# 가중치 가지치기 및 양자화를 통한 모델 축소

## 요약

1. MNIST 데이터를 tf.keras 라이브러리를 이용해 모델링한다.
2. Pruning API를 적용하여 모델을 미세조정하고 정확도를 확인한다.
3. 가지치기에서 3배 더 작은 TF와 TFLite 모델을 만든다.
4. 가지치기와 훈련 후 양자화를 거쳐 10배 더 작은 TFLite 모델을 만든다.
5. 최적화된 TF 및 TFLite 모델의 정확도를 확인한다.

## 라이브러리 불러오기 및 설정

In [1]:
import os
import zipfile
import tempfile

import numpy as np
import tensorflow as tf

import tensorflow_model_optimization as tfmot

In [2]:
# TensorFlow 로그 제어
from freeman.utils.support_tf import LogLevelManager as llm
llm.set(2)      # 경고 이상만 출력(DEBUG, INFO 제외)

In [9]:
BASE_MODEL_DIR = os.path.join(os.path.expanduser("~"), "temp")
FILE_MODEL_NORMAL = os.path.join(BASE_MODEL_DIR, "model_normal.h5")
FILE_MODEL_KERAS = os.path.join(BASE_MODEL_DIR, "model_keras.h5")
FILE_MODEL_TFLITE = os.path.join(BASE_MODEL_DIR, "model_tflite.tflite")

## 일반 모델 훈련(with MNIST)

In [4]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
train_x.shape, test_x.shape

((60000, 28, 28), (10000, 28, 28))

In [5]:
# 정규화(0~1)
train_x, test_x = train_x / 255., test_x / 255.

In [6]:
model_normal = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=12, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

model_normal.compile(optimizer="adam",
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=["accuracy"])

model_normal.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 12)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
____________________________________________________

In [8]:
%%time
# 모델 훈련
model_normal.fit(train_x, train_y, epochs=5, validation_split=0.1, verbose=0)

CPU times: user 1min 17s, sys: 10.8 s, total: 1min 28s
Wall time: 50.7 s


In [10]:
# 모델 평가
_, accuracy_normal = model_normal.evaluate(test_x, test_y)

313/313 [==============================] - 2s 5ms/step - loss: 0.0684 - accuracy: 0.9782


In [11]:
# 모델 저장
tf.keras.models.save_model(model_normal, FILE_MODEL_NORMAL, include_optimizer=False)
size_file_normal = os.path.getsize(FILE_MODEL_NORMAL)

## 가지치기(Pruning)를 통한 모델 미세조정

* 가지치기를 전체 모델에 적용하고 모델 요약에서 이를 확인한다.
* 이 예제는 50% 희소성(가중치가 0인 50%)으로 모델을 시작하고, 80% 희소성으로 종료한다.
* <b>모델 정확도를 높이기 위해 일부 레이어를 잘라낼 수 도 있다.</b>(이 예제에는 없음)

In [ ]:
# Pruning 옵션 설정

## 여기서 정의된 batch_size, epochs 등은 위에서 학습된 모델에서 사용된 것이 아니라,
## 가지치기를 위한 파라미터임.
batch_size = 128
epochs = 2
validation_split = 0.1

num_train_data = train_x.shape[0] * (1 - validation_split)
